In [6]:
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chira\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
import re
import pandas as pd
import numpy as np
from pprint import pprint
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import gensim
import pyLDAvis
import pyLDAvis.sklearn
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
pyLDAvis.enable_notebook()
%matplotlib inline
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import spacy

C:\Users\chira\AppData\Local\Programs\Python\Python38\lib\site-packages\matplotlib_inline\config.py:66: DeprecationWarning: InlineBackend._figure_formats_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _figure_formats_changed(self, name, old, new):


In [8]:
data = pd.read_csv('intern_data.csv')
print(data.new_genre.unique())
data.head()

['happy' 'romantic' 'folk' 'classical' 'dance' 'rock' 'ghazals' 'bhakti'
 'sad' 'others' 'pop']


,lyrics,composers,singers,lyricist,music_director,cast,movie_director,music_label,movie,year,source,old_genre,new_genre,new_composer
0,tan ke tambure me do saso ke tar bole jai radh...,NaN,Manna Dey,Bharat Vyas,S N Tripathi,"B M Vyas, Manhar Desai, Nirupa Roy, Krishan Ku...","Manno Desai, Vijay Sadanah",Sunil Goel,Janam Janam Ke Phere,1957.0,https://joshlyrics.com/hindi_movie/janam-janam...,Playful Songs,happy,S N Tripathi
1,o hansani meri hansani; kaha ud chali mere arm...,NaN,Kishore Kumar,Majrooh Sultanpuri,R D Burman,"Rishi Kapoor, Moushmi Chatterjee, Neetu Singh,...",S R Puttanna Kanagal,Virendra Sinha,Zehreela Insaan,1974.0,https://joshlyrics.com/hindi_movie/zehreela-in...,Playful Songs |Romantic Songs,romantic,R D Burman
2,saanso ki jarurat hai jaise saanso ki jarurat ...,NaN,Anuradha Paudwal,Sameer,"Nadeem Saifee, Shravan","Rahul Roy, Anu Agarwal, Reema Lagoo, Deepak Ti...",Mahesh Bhatt,"Gulshan Kumar, Mukesh Bhatt",Aashiqui,1990.0,https://joshlyrics.com/hindi_movie/aashiqui/ek...,Romantic Songs,romantic,"Nadeem Saifee,Shravan"
3,nafarat ki laathi todo laalach kaa khajar phek...,NaN,Mohammed Rafi,Anand Bakshi,"Laxmikant Kudalkar, Pyarelal","Amitabh Bachchan, Hema Malini, Amjad Khan, Sha...","Kawal Sharma, Latif Khan, Manmohan Desai, Nare...",Subhash Desai,Desh Premee,1982.0,https://joshlyrics.com/hindi_movie/desh-premee...,Patriotic Songs,folk,"Laxmikant Kudalkar,Pyarelal"
4,kaha hai unhone ye raaz e mohabbat kasam hai t...,NaN,Lata Mangeshkar,Satyendra,Roshan,"Raj Kapoor, Nargis Dutt, Agha, Achla Sachdev, ...",K A Abbas,Naya Sansar,Anhonee,1952.0,https://joshlyrics.com/hindi_movie/anhonee/kah...,Qawwalis,classical,Roshan


In [30]:
data.drop(['singers','movie','source','old_genre','movie_director','music_label','cast','music_director','new_composer'],axis = 1, inplace = True)
data

KeyError: "['singers' 'movie' 'source' 'old_genre' 'movie_director' 'music_label'\n 'cast' 'music_director' 'new_composer'] not found in axis"

In [9]:
data.drop(['composers'], inplace = True, axis = 1)
data

,lyrics,singers,lyricist,music_director,cast,movie_director,music_label,movie,year,source,old_genre,new_genre,new_composer
0,tan ke tambure me do saso ke tar bole jai radh...,Manna Dey,Bharat Vyas,S N Tripathi,"B M Vyas, Manhar Desai, Nirupa Roy, Krishan Ku...","Manno Desai, Vijay Sadanah",Sunil Goel,Janam Janam Ke Phere,1957.0,https://joshlyrics.com/hindi_movie/janam-janam...,Playful Songs,happy,S N Tripathi
1,o hansani meri hansani; kaha ud chali mere arm...,Kishore Kumar,Majrooh Sultanpuri,R D Burman,"Rishi Kapoor, Moushmi Chatterjee, Neetu Singh,...",S R Puttanna Kanagal,Virendra Sinha,Zehreela Insaan,1974.0,https://joshlyrics.com/hindi_movie/zehreela-in...,Playful Songs |Romantic Songs,romantic,R D Burman
2,saanso ki jarurat hai jaise saanso ki jarurat ...,Anuradha Paudwal,Sameer,"Nadeem Saifee, Shravan","Rahul Roy, Anu Agarwal, Reema Lagoo, Deepak Ti...",Mahesh Bhatt,"Gulshan Kumar, Mukesh Bhatt",Aashiqui,1990.0,https://joshlyrics.com/hindi_movie/aashiqui/ek...,Romantic Songs,romantic,"Nadeem Saifee,Shravan"
3,nafarat ki laathi todo laalach kaa khajar phek...,Mohammed Rafi,Anand Bakshi,"Laxmikant Kudalkar, Pyarelal","Amitabh Bachchan, Hema Malini, Amjad Khan, Sha...","Kawal Sharma, Latif Khan, Manmohan Desai, Nare...",Subhash Desai,Desh Premee,1982.0,https://joshlyrics.com/hindi_movie/desh-premee...,Patriotic Songs,folk,"Laxmikant Kudalkar,Pyarelal"
4,kaha hai unhone ye raaz e mohabbat kasam hai t...,Lata Mangeshkar,Satyendra,Roshan,"Raj Kapoor, Nargis Dutt, Agha, Achla Sachdev, ...",K A Abbas,Naya Sansar,Anhonee,1952.0,https://joshlyrics.com/hindi_movie/anhonee/kah...,Qawwalis,classical,Roshan
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25885,ya ya ya o ya yea ya ya oo yeye ya oho o oho o...,Mohit Chauhan,Irshad Kamil,A R Rahman,"Ranbir Kapoor, Nargis Fakhri, Shammi Kapoor, A...",Imtiaz Ali,Dhillin Mehta,Rockstar,2011.0,https://joshlyrics.com/hindi_movie/rockstar/jo...,Motivational Songs,others,A R Rahman
25886,ÿþjhuthi sachi aas pe jina kab tak aakhir; aak...,"Chitra Singh, Jagjit Singh",NaN,NaN,NaN,NaN,NaN,Beyond Time (Jagjit Singh),1988.0,https://joshlyrics.com/hindi_movie/beyond-time...,Ghazals,ghazals,NaN
25887,mere yaro mere aane ki mere yaro mere aane ki ...,Shailendra Singh,Anand Bakshi,"Laxmikant Kudalkar, Pyarelal","Sanjay Dutt, Rati Agnihotri, Jaya Prada, Kader...",S V Rajendra Singh,Ranjit Virk,Mera Faisla,1984.0,https://joshlyrics.com/hindi_movie/mera-faisla...,Playful Songs,happy,"Laxmikant Kudalkar, Pyarelal"
25888,ai dil divane ai dil divane ai dil divane ai d...,Alka Yagnik,Rani Malik,"Anand Raj Anand, Pradeep Laad, Naeem Ejaz","Jackie Shroff, Kajol, Atul Agnihotri, Ayesha J...",Feroze Irani,"Ashutosh Bajpay, Pradeep Jain, Saumil Patel",Hote Hote Pyaar Ho Gaya,1999.0,https://joshlyrics.com/hindi_movie/hote-hote-p...,Romantic Songs,romantic,"Anand Raj Anand, Pradeep Laad, Naeem Ejaz"


In [10]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data.lyrics))

print(data_words[:1])

[['tan', 'ke', 'tambure', 'me', 'do', 'saso', 'ke', 'tar', 'bole', 'jai', 'radhe', 'shyam', 'shyam', 'jai', 'sita', 'ram', 'ram', 'tan', 'ke', 'tambure', 'me', 'do', 'saso', 'ke', 'tar', 'bole', 'jai', 'radhe', 'shyam', 'shyam', 'jai', 'sita', 'ram', 'ram', 'ab', 'to', 'is', 'tan', 'ke', 'mandir', 'me', 'parbhu', 'ka', 'hua', 'basera', 'parbhu', 'ka', 'hua', 'basera', 'ab', 'to', 'is', 'tan', 'ke', 'mandir', 'me', 'parbhu', 'ka', 'hua', 'basera', 'parbhu', 'ka', 'hua', 'basera', 'magan', 'hua', 'man', 'mera', 'magan', 'hua', 'man', 'mera', 'chhuta', 'janam', 'janam', 'ka', 'fera', 're', 'janam', 'janam', 'ka', 'fera', 'man', 'ko', 'muraliya', 'me', 'man', 'ko', 'muraliya', 'me', 'man', 'ko', 'muraliya', 'me', 'sur', 'ka', 'singar', 'bole', 'jai', 'radhe', 'shyam', 'shyam', 'jai', 'sita', 'ram', 'ram', 'tan', 'ke', 'tambure', 'me', 'do', 'saso', 'ke', 'tar', 'bole', 'jai', 'radhe', 'shyam', 'shyam', 'jai', 'sita', 'ram', 'ram', 'lagi', 'lagan', 'lila', 'dhari', 'se', 'jagi', 're', 'jagm

In [40]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out
    


In [2]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

NameError: name 'spacy' is not defined

In [11]:

from gensim import corpora
id2word = corpora.Dictionary(data_words)
texts = data_words
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[:1])


[[(0, 1), (1, 2), (2, 3), (3, 4), (4, 6), (5, 1), (6, 1), (7, 2), (8, 7), (9, 2), (10, 2), (11, 6), (12, 2), (13, 2), (14, 3), (15, 3), (16, 14), (17, 4), (18, 3), (19, 8), (20, 10), (21, 1), (22, 3), (23, 2), (24, 2), (25, 2), (26, 2), (27, 5), (28, 2), (29, 12), (30, 2), (31, 1), (32, 3), (33, 3), (34, 4), (35, 2), (36, 3), (37, 7), (38, 16), (39, 4), (40, 4), (41, 2), (42, 15), (43, 1), (44, 7), (45, 1), (46, 4), (47, 6), (48, 4), (49, 2)]]


In [12]:
id2word[0]

'aasu'

In [13]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('aasu', 1),
  ('ab', 2),
  ('ankhiyo', 3),
  ('basera', 4),
  ('bole', 6),
  ('chhuta', 1),
  ('dhar', 1),
  ('dhari', 2),
  ('do', 7),
  ('fera', 2),
  ('heera', 2),
  ('hua', 6),
  ('is', 2),
  ('ja', 2),
  ('jagi', 3),
  ('jagmag', 3),
  ('jai', 14),
  ('janam', 4),
  ('jyoti', 3),
  ('ka', 8),
  ('ke', 10),
  ('ki', 1),
  ('ko', 3),
  ('lagan', 2),
  ('lagi', 2),
  ('lila', 2),
  ('magan', 2),
  ('man', 5),
  ('mandir', 2),
  ('me', 12),
  ('mera', 2),
  ('moti', 1),
  ('muraliya', 3),
  ('nam', 3),
  ('parbhu', 4),
  ('paya', 2),
  ('pyasi', 3),
  ('radhe', 7),
  ('ram', 16),
  ('re', 4),
  ('saso', 4),
  ('se', 2),
  ('shyam', 15),
  ('singar', 1),
  ('sita', 7),
  ('sur', 1),
  ('tambure', 4),
  ('tan', 6),
  ('tar', 4),
  ('to', 2)]]

In [21]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=50, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [22]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]


[(18,
  '0.111*"ni" + 0.086*"on" + 0.079*"it" + 0.068*"baby" + 0.061*"and" + '
  '0.046*"we" + 0.038*"be" + 0.038*"your" + 0.037*"just" + 0.037*"kudiye"'),
 (12,
  '0.251*"he" + 0.218*"jan" + 0.081*"tab" + 0.053*"ganga" + 0.047*"jage" + '
  '0.039*"jay" + 0.038*"bharat" + 0.035*"gaaye" + 0.022*"devi" + '
  '0.017*"uchhal"'),
 (32,
  '0.160*"ma" + 0.134*"suno" + 0.078*"moti" + 0.060*"rahu" + 0.059*"maiya" + '
  '0.056*"isko" + 0.047*"bhag" + 0.037*"sathiya" + 0.033*"pari" + '
  '0.032*"matwala"'),
 (38,
  '0.202*"the" + 0.126*"you" + 0.119*"di" + 0.098*"love" + 0.084*"baba" + '
  '0.064*"da" + 0.049*"dam" + 0.039*"ruk" + 0.030*"naal" + 0.029*"nu"'),
 (21,
  '0.262*"lage" + 0.204*"pyaar" + 0.131*"jaane" + 0.075*"yun" + 0.068*"hasin" '
  '+ 0.060*"karke" + 0.046*"aayega" + 0.028*"kinara" + 0.026*"badla" + '
  '0.017*"vahi"'),
 (47,
  '0.158*"jai" + 0.145*"ram" + 0.079*"jag" + 0.077*"janam" + 0.073*"bolo" + '
  '0.067*"tan" + 0.043*"mann" + 0.028*"sagar" + 0.027*"shri" + 0.025*"gagan"'),
 

In [23]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

C:\Users\chira\AppData\Local\Programs\Python\Python38\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
43    -0.278346 -0.165692       1        1  11.028174
44    -0.232827 -0.087822       2        1   9.876044
40    -0.145358  0.159274       3        1   5.163926
7     -0.150279  0.070406       4        1   5.021949
19    -0.202509 -0.195405       5        1   4.864420
36    -0.111822  0.103834       6        1   4.775077
4     -0.035768  0.098585       7        1   3.380096
3     -0.075810  0.105701       8        1   2.950587
1     -0.068477  0.029596       9        1   2.611395
29    -0.025058  0.100375      10        1   2.487664
24    -0.017267  0.067297      11        1   2.282784
30    -0.048211  0.108090      12        1   2.209771
22    -0.031500  0.036964      13        1   2.140709
16    -0.096049 -0.129015      14        1   2.009564
45     0.000846  0.080685      15        1   1.851291
28    -0.001263  0.063960      16        1   1.678829
9     -0.052952 -0.052848      17        1   1.607719
10     0.005621  0.056903      18        1   1.593672
23     0.004000  0.053177      19        1   1.534735
25     0.000406  0.077357      20        1   1.531654
39     0.009115 -0.002122      21        1   1.437822
41     0.014420  0.036539      22        1   1.415536
5      0.027559  0.024125      23        1   1.413334
8      0.018166  0.051910      24        1   1.322327
35     0.036864 -0.030626      25        1   1.279169
6      0.029767  0.016123      26        1   1.253343
15     0.019407  0.004115      27        1   1.237650
13     0.034855  0.010687      28        1   1.208100
14     0.033767  0.006695      29        1   1.207983
33     0.041723  0.003789      30        1   1.168472
48     0.025240 -0.006653      31        1   1.138689
11     0.039964  0.007060      32        1   1.133516
37     0.034160  0.012826      33        1   1.111330
0      0.045861  0.010672      34        1   1.100038
38     0.052573 -0.004542      35        1   1.054473
26     0.054946 -0.011914      36        1   1.013252
20     0.031256 -0.057154      37        1   1.008236
46     0.052512  0.000416      38        1   0.929890
17     0.030026 -0.045659      39        1   0.929823
31     0.058790 -0.021603      40        1   0.908404
47     0.061462 -0.012882      41        1   0.887947
49     0.075318 -0.034320      42        1   0.752766
42     0.075519 -0.029475      43        1   0.713030
34     0.084511 -0.045159      44        1   0.687532
2      0.078860 -0.037994      45        1   0.657376
18     0.050344 -0.164637      46        1   0.618155
21     0.095013 -0.053636      47        1   0.582250
27     0.092277 -0.046304      48        1   0.578689
12     0.130516 -0.082036      49        1   0.326349
32     0.127832 -0.079667      50        1   0.324459, topic_info=        Term           Freq          Total Category  logprob  loglift
100      hai  284340.000000  284340.000000  Default  30.0000  30.0000
29        me  118870.000000  118870.000000  Default  29.0000  29.0000
193       na  110320.000000  110320.000000  Default  28.0000  28.0000
21        ki   98448.000000   98448.000000  Default  27.0000  27.0000
20        ke  110151.000000  110151.000000  Default  26.0000  26.0000
..       ...            ...            ...      ...      ...      ...
57   hansani       0.022731       1.280745  Topic50 -13.5693   1.6993
0       aasu       0.022731     973.040090  Topic50 -13.5693  -4.9337
1         ab       0.022731   20119.870139  Topic50 -13.5693  -7.9627
2    ankhiyo       0.022731    1099.974722  Topic50 -13.5693  -5.0563
3     basera       0.022731     358.291945  Topic50 -13.5693  -3.9346

[1873 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
423      30  0.999939       aa
1546     21  0.997573    aadhi
2076     38  0.998603    aadmi
472      15  0.999681      aag
751       1  0.012045     aage
...     ...       ...      ...
1349     29  0.999103  

In [20]:
import gc
del lda_model
gc.collect

<function gc.collect(generation=2)>